In [74]:
import os
import gc
import glob
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
import colorcet  # Extra colormaps

In [75]:
import matplotlib.pyplot as plt
import colorcet  # Extra colormaps

# Define available colormaps
COLORMAPS = {
    # 🌎 Extra Colorcet Colormaps (for better color mapping)
    "rainbow4": colorcet.cm.rainbow4,  # Improved rainbow
    
    # # 🌈 Rainbow & Jet Variants (Not Uniform, but Classic)
    # "jet": plt.cm.jet,  # Classic rainbow
    # "rainbow": plt.cm.rainbow,  # Full spectrum
    # "nipy_spectral": plt.cm.nipy_spectral,  # Alternative rainbow
}

DEFAULT_COLORMAP = "rainbow4"
NDVI_MIN, NDVI_MAX = 0.5, 1.0  # NDVI range

In [76]:
def normalize_ndvi(ndvi_array, vmin=NDVI_MIN, vmax=NDVI_MAX, nodata_value=None):
    """Normalize NDVI values to 0-1 scale, handling NoData values."""
    ndvi_array = ndvi_array.astype(np.float32)  # Ensure float calculations
    
    # Replace nodata values with NaN
    if nodata_value is not None:
        ndvi_array[ndvi_array == nodata_value] = np.nan

    # Normalize (ignore NaNs)
    normalized = (ndvi_array - vmin) / (vmax - vmin)
    normalized = np.clip(normalized, 0, 1)  # Keep values in range

    return normalized

In [77]:
def apply_colormap(image_array, colormap):
    """Apply colormap to normalized NDVI and return as RGB (0-255)."""
    normalized = normalize_ndvi(image_array)
    colored = colormap(normalized)[:, :, :3]  # Remove alpha channel
    return (colored * 255).astype(np.uint8)

In [78]:
def process_ndvi_tif(file_path, output_path, colormap_name):
    """Read NDVI TIFF, handle nodata, colorize it, and save as new TIFF with color bar."""
    print(f"\nProcessing: {os.path.basename(file_path)} with {colormap_name}")
    
    try:
        with rasterio.open(file_path) as src:
            ndvi = src.read(1).astype(np.float32)  # Read as float to handle NaNs
            nodata_value = src.nodata  # Get NoData value from metadata

            # Apply selected colormap
            colormap = COLORMAPS[colormap_name]
            rgb_array = apply_colormap(ndvi, colormap)

            # Normalize the NDVI for the color bar
            norm = plt.Normalize(vmin=NDVI_MIN, vmax=NDVI_MAX)
            sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
            sm.set_array([])

            # Create the figure and axis for displaying the color bar with the image
            fig, ax = plt.subplots(figsize=(12, 8))
            ax.imshow(rgb_array)
            cbar = fig.colorbar(sm, ax=ax, orientation="vertical", fraction=0.02, pad=0.04)
            cbar.set_label("NDVI Value")

            # Update metadata for RGB output
            profile = src.profile
            profile.update(
                dtype=rasterio.uint8,
                count=3,  # Three bands for RGB
                photometric="RGB",
                nodata=None  # Remove NoData for RGB
            )

            # Save the image with the color bar
            output_colormap_path = output_path.replace(".tif", f"_{colormap_name}.tif")

            print(f" - Saving colorized TIFF: {output_colormap_path}")
            with rasterio.open(output_colormap_path, "w", **profile) as dst:
                for i in range(3):
                    dst.write(rgb_array[:, :, i], i + 1)

            print(f"✅ Successfully processed: {os.path.basename(file_path)} with {colormap_name}")

            # Save the image with the color bar as an image file (for visualization)
            colorized_image_path = output_path.replace(".tif", f"_{colormap_name}_with_colorbar.png")
            plt.savefig(colorized_image_path, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            del ndvi, rgb_array
            gc.collect()
            
            print(f" - Colorized image with color bar saved as: {colorized_image_path}")

    except Exception as e:
        print(f"❌ ERROR processing {file_path}: {str(e)}")

In [ ]:
def process_orthomosaics_with_colormap(orthomosaic_files, colormap_dict, output_folder):
    """Process each orthomosaic with the selected colormaps and save to subfolders with color bar."""
    for key_word, field_dict in orthomosaic_files.items():
        for field_name, file_list in field_dict.items():
            print(f"\nProcessing orthomosaics for field: {field_name} under keyword: {key_word}")
            
            for colormap_name in colormap_dict:
                colormap_folder = os.path.join(output_folder, field_name, key_word, colormap_name)
                os.makedirs(colormap_folder, exist_ok=True)
                
                for file_path in file_list:
                    output_path = os.path.join(colormap_folder, os.path.basename(file_path))
                    process_ndvi_tif(file_path, output_path, colormap_name)
            gc.collect()
    print("\n✅ Colorization complete for all images!")

In [79]:
def find_orthomosaic_files(input_folder, search_config):
    """
    Find orthomosaic files based on the path structure and search configuration.
    
    Args:
        input_folder (str): The base directory where field_name/key_word/ files are located.
        search_config (dict): Dictionary with keywords and suffixes to search for.
        
    Returns:
        dict: Dictionary with keyword as keys and field names with their file lists.
    """
    orthomosaic_files = {}

    # Walk through the field_name folders inside input_folder
    for field_name in os.listdir(input_folder):
        field_path = os.path.join(input_folder, field_name)
        
        # Skip if not a directory
        if not os.path.isdir(field_path):
            continue
        
        for key_word, (suffix, _) in search_config.items():
            # Search for files under the key_word folder
            key_word_folder = os.path.join(field_path, key_word)

            # If the key_word folder exists, search for orthomosaics
            if os.path.isdir(key_word_folder):
                file_paths = glob.glob(os.path.join(key_word_folder, f'*{suffix}'))
                if file_paths:
                    if key_word not in orthomosaic_files:
                        orthomosaic_files[key_word] = {}
                    orthomosaic_files[key_word][field_name] = file_paths

    return orthomosaic_files


In [86]:
# Example usage:

# Define the input folder where the projects and orthomosaics are stored
input_folder = r"D:\PhenoCrop\3_orthomosaics_rgb_ndvi"

# Example search configuration to find TIFF or JPG files
search_config = {
    "NDVI": ("_index_ndvi.tif", _), # TIFF files scaled to 50%
}

# Define the output folder to save colorized files
output_folder = r"D:\PhenoCrop\NDVI_rainbow4_"+str(NDVI_MIN)+"-"+str(NDVI_MAX)

# Rotation angles for specific fields (not used directly here but could be added in the future)
rotation_dict = {
     'PHENO_CROP': -13.5,
     'OAT_FRONTIERS': -13.5,
     'DIVERSITY_OATS': -13.5,
     'PRO_BAR_VOLL': -29.5,
     'E166': -115.5,
     'PRO_BAR_SØRÅS': -68.5
}

# Find orthomosaic files
orthomosaic_files = find_orthomosaic_files(input_folder, search_config)
# orthomosaic_files

In [82]:
inner_keys = {inner_key for inner_dict in orthomosaic_files.values() if isinstance(inner_dict, dict) for inner_key in inner_dict.keys()}
print("All Inner Keys:", inner_keys)

All Inner Keys: {'OAT_FRONTIERS', 'PILOT', 'E166', 'DIVERSITY_OATS', 'PHENO_CROP', 'PRO_BAR_VOLL'}


In [83]:
# def delete_inner_keys(nested_dict, field_name, key_word):
#     """Deletes a specific inner key (key_word) from a nested dictionary (field_name level)."""
#     if field_name in nested_dict and key_word in nested_dict[field_name]:
#         del nested_dict[field_name][key_word]
#     #     print(f"Deleted key '{key_word}' from '{field_name}'")
#     # else:
#     #     print(f"Key '{key_word}' not found in '{field_name}'")

# delete_fields = [ "PILOT", "PHENO_CROP", "OAT_FRONTIERS", "E166", "DIVERSITY_OATS", "PRO_BAR_VOLL"]

# for field in delete_fields:
#     delete_inner_keys(orthomosaic_files, "NDVI", field)

In [84]:
orthomosaic_files

{'NDVI': {'DIVERSITY_OATS': ['D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240603 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240607 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240611 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240619 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240624 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240703 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240708 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\DIVERSITY_OATS\\NDVI\\20240718 DIV

In [85]:
# Process the images with colorization and save them in corresponding colormap subfolders
process_orthomosaics_with_colormap(orthomosaic_files, COLORMAPS, output_folder)


Processing orthomosaics for field: DIVERSITY_OATS under keyword: NDVI

Processing: 20240603 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif with rainbow4
 - Saving colorized TIFF: D:\PhenoCrop\NDVI_rainbow4_0.5-1.0\DIVERSITY_OATS\NDVI\rainbow4\20240603 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4.tif
✅ Successfully processed: 20240603 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif with rainbow4
 - Colorized image with color bar saved as: D:\PhenoCrop\NDVI_rainbow4_0.5-1.0\DIVERSITY_OATS\NDVI\rainbow4\20240603 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4_with_colorbar.png

Processing: 20240607 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif with rainbow4
 - Saving colorized TIFF: D:\PhenoCrop\NDVI_rainbow4_0.5-1.0\DIVERSITY_OATS\NDVI\rainbow4\20240607 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%_rainbow4.tif
✅ Successfully processed: 20240607 DIVERSITY_OATS M3M 30m MS 80 85_index_ndvi_50%.tif with rainbow4
 - Colorized image with color bar saved as: D:

# Filter Orthomosaic Files Dict

In [52]:
import pprint

def filter_orthomosaic_files(orthomosaic_files, substrings):
    """Filter file paths in a nested dictionary based on multiple substrings."""
    filtered_files = {}

    for category, sub_dict in orthomosaic_files.items():
        filtered_sub_dict = {}

        for sub_category, file_list in sub_dict.items():
            print(f"\nProcessing category: {category}, sub-category: {sub_category}")

            # Keep files that contain ANY of the substrings
            filtered_list = [file for file in file_list if any(sub in file for sub in substrings)]


            if filtered_list:  # Only keep non-empty lists
                filtered_sub_dict[sub_category] = filtered_list
        if filtered_sub_dict:  # Only keep non-empty categories
            filtered_files[category] = filtered_sub_dict


    return filtered_files

In [53]:
orthomosaic_files = find_orthomosaic_files(input_folder, search_config)
delete_inner_keys(orthomosaic_files, "NDVI", "PILOT")
delete_inner_keys(orthomosaic_files, "NDVI", "PHENO_CROP")
delete_inner_keys(orthomosaic_files, "NDVI", "OAT_FRONTIERS")
delete_inner_keys(orthomosaic_files, "NDVI", "E166")
delete_inner_keys(orthomosaic_files, "NDVI", "DIVERSITY_OATS")
delete_inner_keys(orthomosaic_files, "NDVI", "PRO_BAR_VOLL")
orthomosaic_files

Deleted key 'PILOT' from 'NDVI'
Deleted key 'PHENO_CROP' from 'NDVI'
Deleted key 'OAT_FRONTIERS' from 'NDVI'
Deleted key 'E166' from 'NDVI'
Deleted key 'DIVERSITY_OATS' from 'NDVI'
Deleted key 'PRO_BAR_VOLL' from 'NDVI'


{'NDVI': {'PRO_BAR_SØRÅS': ['D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240523 PRO_BAR_SØRÅS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240604 PRO_BAR_SØRÅS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240606 PRO_BAR_SØRÅS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240611 PRO_BAR_SØRÅS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240620 PRO_BAR_SØRÅS M3M 30m MS 70 70_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240620 PRO_BAR_SØRÅS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240621 PRO_BAR_SØRÅS M3M 30m MS 70 70_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240621 PRO_BAR_SØRÅS M3M 3

In [54]:
# Example list of substrings to filter by
substrings_to_filter = ['20240604', '20240708', '20240807' ]

# Apply filtering
filtered_result = filter_orthomosaic_files(orthomosaic_files, substrings_to_filter)

# Print the filtered result
filtered_result


Processing category: NDVI, sub-category: PRO_BAR_SØRÅS


{'NDVI': {'PRO_BAR_SØRÅS': ['D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240604 PRO_BAR_SØRÅS M3M 30m MS 80 85_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240708 PRO_BAR_SØRÅS M3M 30m MS 70 75_index_ndvi_50%.tif',
   'D:\\PhenoCrop\\3_orthomosaics_rgb_ndvi\\PRO_BAR_SØRÅS\\NDVI\\20240807 PRO_BAR_SØRÅS M3M 30m MS 70 75_index_ndvi_50%.tif']}}

In [ ]:
import matplotlib.pyplot as plt
import colorcet  # Extra colormaps

COLORMAPS = {
    "jet": plt.cm.jet,  # Classic rainbow
    "RdYlGn": plt.cm.RdYlGn,  # Red → Yellow → Green    
    # 🔥 Perceptually Uniform (Best for Scientific Visualization)
    "plasma": plt.cm.plasma,  # Purple → Yellow
    "inferno": plt.cm.inferno,  # Dark Blue → Orange
    "magma": plt.cm.magma,  # Dark Blue → Yellow
    "cividis": plt.cm.cividis,  # Blue → Green (colorblind-friendly)
    "viridis": plt.cm.viridis,  # Default, blue → yellow

    # 🎨 Classic Diverging (Good for Differences)
    "coolwarm": plt.cm.coolwarm,  # Blue → White → Red
    "RdYlBu": plt.cm.RdYlBu,  # Red → Yellow → Blue
    # "RdYlGn": plt.cm.RdYlGn,  # Red → Yellow → Green
    "PiYG": plt.cm.PiYG,  # Pink → Yellow → Green
    "Spectral": plt.cm.Spectral,  # Multiple colors
    "bwr": plt.cm.bwr,  # Blue → White → Red

    # 🔵 Sequential Colormaps (For NDVI, Elevation, etc.)
    "Blues": plt.cm.Blues,  # Light Blue → Dark Blue
    "Greens": plt.cm.Greens,  # Light Green → Dark Green
    "Oranges": plt.cm.Oranges,  # Light Orange → Dark Orange
    "Purples": plt.cm.Purples,  # Light Purple → Dark Purple
    "Reds": plt.cm.Reds,  # Light Red → Dark Red

    # 🌈 Rainbow & Jet Variants (Not Uniform, but Classic)

    "rainbow": plt.cm.rainbow,  # Full spectrum
    "nipy_spectral": plt.cm.nipy_spectral,  # Alternative rainbow

    # 🌕 Cyclic (Best for Wind Direction, Phase Angles)
    "twilight": plt.cm.twilight,  # Smooth transition
    "twilight_shifted": plt.cm.twilight_shifted,  # Same but shifted
    "hsv": plt.cm.hsv,  # Hue cycle

    # 🔴🔵 Categorical (For Class Labels)
    "tab10": plt.cm.tab10,  # 10 bold colors
    "tab20": plt.cm.tab20,  # 20 colors
    "Set1": plt.cm.Set1,  # High contrast
    "Set2": plt.cm.Set2,  # Softer colors
    "Set3": plt.cm.Set3,  # More variety

    # 🌎 Extra Colorcet Colormaps (for better color mapping)
    "fire": colorcet.cm.fire,  # Dark → Bright Red/Orange (like magma)
    "glasbey": colorcet.cm.glasbey,  # Discrete, colorblind-safe
    "rainbow4": colorcet.cm.rainbow4,  # Improved rainbow
    "kb": colorcet.cm.kbc,  # Black → Blue → Cyan
    "kr": colorcet.cm.kr,  # Black → Red → Yellow
    "bgy": colorcet.cm.bgy,  # Blue → Green → Yellow
    "bmw": colorcet.cm.bmw,  # Blue → Magenta → White
    "coolwarm_cet": colorcet.cm.coolwarm,  # Better coolwarm variant
}
